In [1]:
from tqdm import tqdm
from nltk import bigrams
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score, precision_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import preprocessing
import keras
from keras.utils import np_utils
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, BatchNormalization, Dropout, Input, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [2]:
gloveEmbeddings = {}
f = open('../data/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    gloveEmbeddings[word] = coefs
f.close()

In [3]:
DATA_PATH = './../data/Brown_tagged_train.txt'


In [4]:
with open(DATA_PATH, 'r') as f:
    data = f.read().splitlines()
data = np.array(data)


In [5]:
def split_Xy(test_Xy):
    """
    test_Xy: List of list of tokens and tags
    Returns: List of tokens and list of tags
    """
    test_y = []
    test_X = []

    for sent in test_Xy:
        tagged_sent = sent.split(' ')
        sent_y = []
        sent_X = []

        for word in tagged_sent:
            if word == "":
                continue
            actual_word, tag = split_tag_word(word)
            sent_X.append(actual_word)
            sent_y.append(tag)

        test_y.append(sent_y)
        test_X.append(sent_X)

    return test_X, test_y


In [6]:
def split_tag_word(inp):
    """
    Returns word, tag for the given input
    """
    arr = inp.split('/')
    tag = arr[-1]
    del arr[-1]
    word = '/'.join(arr)
    return word, tag


In [7]:
trainSet, testXy = split_Xy(data)


In [8]:
def transform_to_dataset(sentences, tags, window=1):
    X = []
    y = []
    unk = 0
    for ctr in tqdm(range(len(sentences))):
        sent = sentences[ctr]
        for i in range(len(sent)):
            vec, unknown = features_embs(sent, i, window)
            X.append(vec)
            y.append(tags[ctr][i])
            unk += unknown
    return X, y, unk


def vectorize(trainSent, trainTags, window=1, embedding='word2vec'):
    X, y, unk = transform_to_dataset(trainSent, trainTags, window)
    # X = np.array(X)
    # y = np.array(y)
    print('Unknown words:', unk)
    X = np.asarray(X, dtype=object)
    y = np.asarray(y, dtype=object)
    print('X shape:', X.shape)
    print('y shape:', y.shape)
    return X, y


def getYtrain(sentences, tags):
    y_train = []
    for ctr in tqdm(range(len(sentences))):
        sent = sentences[ctr]
        for i in range(len(sent)):
            y_train.append(tags[ctr][i])
    y_train = np.array(y_train)
    return y_train


In [9]:
y_train = getYtrain(trainSet, testXy)


100%|██████████| 27491/27491 [00:00<00:00, 95873.97it/s]


In [10]:
classes = sorted(list(set(y_train)))


In [11]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_train = np_utils.to_categorical(y_train)

print(y_train.shape)


(543149, 12)


In [12]:
embs = []
out_ = 0
in_ = 0
for i in tqdm(range(len(trainSet))):
    for j in trainSet[i]:
        try:
            embs.append(embeddings[j])
            in_ += 1
        except:
            oov = np.random.uniform(-0.25, 0.25, 300)
            embs.append(oov)
            out_ += 1


100%|██████████| 27491/27491 [00:22<00:00, 1205.76it/s]


In [13]:
from sklearn.model_selection import train_test_split
embs_train1, embs_test1, y_train1, y_test1 = train_test_split(
    embs, y_train, test_size=0.2, random_state=42)
print("Previous: ", len(embs), y_train.shape)
print("Current: ", len(embs_train1), len(
    embs_test1), y_train1.shape, y_test1.shape)


Previous:  543149 (543149, 12)
Current:  434519 108630 (434519, 12) (108630, 12)


In [14]:
embs_train1 = np.array(embs_train1)
embs_test1 = np.array(embs_test1)


In [21]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=embs_train1.shape[1]))
# model.add(Dense(300, activation='relu'))
# model.add(Dense(150, activation='relu'))
model.add(Dense(y_train1.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_11 (Dense)             (None, 12)                1212      
Total params: 31,312
Trainable params: 31,312
Non-trainable params: 0
_________________________________________________________________


In [22]:
nb_epoch = 7
batch_size = 128
cp = ModelCheckpoint(filepath="tagger.h5",
                     monitor='val_acc',
                     save_best_only=True,
                     verbose=1)

tb = TensorBoard(log_dir='./logs',
                 histogram_freq=0,
                 write_graph=True,
                 write_images=True)

early_stopping = EarlyStopping(monitor='val_acc', patience=5)

history = model.fit(embs_train1, y_train1,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=1, validation_data=(embs_test1, y_test1))


2021-10-31 22:37:19.699491: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-31 22:37:19.699525: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-31 22:37:19.700341: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/7
3395/3395 [==============================] - 15s 4ms/step - loss: 2.1754 - accuracy: 0.2202 - val_loss: 2.1657 - val_accuracy: 0.2228
Epoch 2/7
3395/3395 [==============================] - 15s 4ms/step - loss: 2.1626 - accuracy: 0.2221 - val_loss: 2.1667 - val_accuracy: 0.2226
Epoch 3/7
3395/3395 [==============================] - 14s 4ms/step - loss: 2.1580 - accuracy: 0.2223 - val_loss: 2.1687 - val_accuracy: 0.2216
Epoch 4/7
3395/3395 [==============================] - 15s 4ms/step - loss: 2.1526 - accuracy: 0.2229 - val_loss: 2.1723 - val_accuracy: 0.2209
Epoch 5/7
3395/3395 [==============================] - 14s 4ms/step - loss: 2.1472 - accuracy: 0.2236 - val_loss: 2.1761 - val_accuracy: 0.2180
Epoch 6/7
3395/3395 [==============================] - 15s 5ms/step - loss: 2.1426 - accuracy: 0.2250 - val_loss: 2.1801 - val_accuracy: 0.2145
Epoch 7/7
3395/3395 [==============================] - 14s 4ms/step - loss: 2.1385 - accuracy: 0.2256 - val_loss: 2.1837 - val_accuracy: